## PROYECTO SPRINT 10
---

### 1. Descripción del proyecto 
Se ha decidido abrir un pequeño café regentado por robots en Los Ángeles. Si bien el proyecto presenta un potencial prometedor, también conlleva un costo significativo. Por ello, se ha optado por explorar estrategias para atraer inversionistas que compartan el interés en las dinámicas actuales del mercado.

Para fortalecer la propuesta de inversión, se llevará a cabo un estudio de mercado exhaustivo utilizando datos recopilados de fuentes abiertas sobre el panorama de restaurantes en Los Ángeles. Este análisis permitirá comprender a fondo las tendencias del sector, identificar oportunidades de mercado y delinear una estrategia sólida para el éxito del café.

---
### 2. Carga de dataset y procesamiento de datos 

In [76]:
# Importación de librerías neceserarias
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from plotly import graph_objects as go
import plotly.express as px
import re

In [2]:
#Asignación de variable al dataset
rest_data = pd.read_csv('rest_data_us_upd.csv')

In [3]:
#Visualización de la información del dataset
rest_data.info()
rest_data.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9651 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9651 non-null   int64 
 1   object_name  9651 non-null   object
 2   address      9651 non-null   object
 3   chain        9648 non-null   object
 4   object_type  9651 non-null   object
 5   number       9651 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 452.5+ KB


,id,object_name,address,chain,object_type,number
872,12658,LA CHINA OAXAQUENA,5710 SANTA MONICA BLVD,False,Restaurant,24
5740,17526,STARBUCKS COFFEE #9227,333 S HOPE ST STE 125-C,True,Cafe,10
2053,13839,MEDITERRANEAN DELIGHT DOWNTOWN,726 S HILL ST,True,Restaurant,33
5478,17264,CHUCK E CHEESE'S #397,2706 WILSHIRE BLVD,True,Restaurant,228
869,12655,SICHA SIAM RESTAURANT,4403 EAGLE ROCK BLVD,True,Restaurant,45
7715,19501,HUANG BBQ & SEAFOOD INC.,4808 MELROSE AVE,True,Restaurant,48
6639,18425,ALAMEDA SNACK BAR,800 MCGARRY ST,False,Bar,19
4448,16234,JACK IN THE BOX #299,4255 E CESAR E CHAVEZ AVE,True,Restaurant,31
6133,17919,88 CHINESE & SUSHI,418 1/2 N FAIRFAX AVE,False,Restaurant,13
1354,13140,THE WAFFLE,6255 W SUNSET BLVD STE #105,False,Restaurant,80


Al analizar el dataset, tanto a través de la visualización de su información como de una muestra representativa, podemos identificar el tipo de datos de cada columna y detectar la presencia de valores ausentes.

En el caso particular de la columna 'chain', observamos que, presenta una baja cantidad de valores ausentes y el tipo de dato asignado no es el adecuado.

Por lo tanto, se procederá a realizar las modificaciones necesarias para garantizar la correcta manipulación y análisis de la información contenida en esta columna 

In [4]:
#Se identifican los valores ausentes de la columna 'chain'
missing_values = rest_data['chain'].isnull()
rows_with_missing_values = rest_data[missing_values]
rows_with_missing_values

,id,object_name,address,chain,object_type,number
7408,19194,TAQUERIA LOS 3 CARNALES,5000 E WHITTIER BLVD,NaN,Restaurant,14
7523,19309,JAMMIN JIMMY'S PIZZA,1641 FIRESTONE BLVD,NaN,Pizza,1
8648,20434,THE LEXINGTON THEATER,129 E 3RD ST,NaN,Restaurant,35


In [5]:
# Cambio de tipo de dato en la columna 'chain'
rest_data['chain'] = rest_data['chain'].astype(bool)

rest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9651 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9651 non-null   int64 
 1   object_name  9651 non-null   object
 2   address      9651 non-null   object
 3   chain        9651 non-null   bool  
 4   object_type  9651 non-null   object
 5   number       9651 non-null   int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 386.5+ KB


Después de realizar el cambio del tipo de dato de de la columna 'chain', se puede observar que ya no existen valores ausentes en la misma columna. 

---
### 3. Análisis de datos 

In [6]:
# Búsqueda de las proporciones de los distintos tipos de establecimientos.
establishments= rest_data.groupby('object_type').agg({'object_name':'count'})
establishments.sort_values(by='object_name', ascending=False)

,object_name
object_type,
Restaurant,7255
Fast Food,1066
Cafe,435
Pizza,320
Bar,292
Bakery,283


In [7]:
#Gráfico para visualizar proporción de los distintos tipos de establecimientos.
labels = establishments.index.to_list()
object_name_values = establishments['object_name'].values
df = px.data.tips()
fig = px.pie(df, values=object_name_values, names=labels, color_discrete_sequence=px.colors.sequential.RdBu, 
             title='Proporciones de los diferentes tipos de establecimientos')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

De acuerdo a la visualización de los resultados a tráves del gráfico, podemos comprobar que el establecimiento tipo 'Restaurant' cuenta con el mayor porcentaje con el 75.2% , seguido de 'Fast Food' con el 11%. Teniendo entre una diferencia bastante significativa. 
El último lugar, lo ocupa 'Bakery' con un 2.93%. 

In [8]:
#Búsqueda de las proporciones de los establecimientos que pertenecen a una cadena y de los que no.
establishments= rest_data.groupby('chain').agg({'object_name':'count'})
establishments

,object_name
chain,
False,5972
True,3679


In [13]:
#Gráfico para visualizar proporción de establecimientos que pertenecen a una cadena.
labels = ['Franquicia','Independiente']
object_name_values = [3679, 5972]
df = px.data.tips()
fig = px.pie(df, values=object_name_values, names=labels, color_discrete_sequence=px.colors.sequential.RdBu, 
             title='Proporción total de establecimientos que pertenecen a una cadena')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

El 61.9% son establecimientos independientes, es decir, no pertenecen a niguna franquicia. El 38.1% pertenecen a una cadena.

In [42]:
# Búsqueda de establecimiento que son habitualmente una cadena.
establishments_chain= rest_data.groupby('object_type').agg({'chain':'sum'})
establishments_chain= establishments_chain.sort_values(by='chain', ascending=False)
establishments_chain

,chain
object_type,
Restaurant,2294
Fast Food,605
Bakery,283
Cafe,266
Pizza,154
Bar,77


In [14]:
# Gráfico para visualizar proporción de establecimientos que pertenecen a una cadena.
labels = establishments_chain.index.to_list()
chain_values = establishments_chain['chain'].values

df = px.data.gapminder()
fig = px.bar(df, y=chain_values, x=labels, text_auto='.2s',
            title="Número de establecimientos que pertenecen a una cadena", 
            color_discrete_sequence=px.colors.sequential.RdBu,
            template='simple_white')

plt.tight_layout()
fig.show()

<Figure size 640x480 with 0 Axes>

Los establecimientos tipo 'Restaurant' pertenecen mayormente a franquicias con 2,294 lugares, por otro lado, el establecimiento tipo 'Bar' es el que menor cantidad de franquicias tiene, con solo 77. 

In [53]:
# ¿Qué caracteriza a las cadenas: muchos establecimientos con un pequeño número de asientos o unos pocos establecimientos con un montón de asientos?

# Encontrar promedio de asientos en establecimientos que son cadena
establishments_chain_seats= rest_data[rest_data['chain'] == True]
print('Número promedio de asientos:', establishments_chain_seats['number'].mean())

# Encontrar número de asientos mayor al promedio 
establishments_chain_seats_= establishments_chain_seats[establishments_chain_seats['number'] > 39]
print('Número de establacimientos con más de 39 asientos:', establishments_chain_seats_['number'].count())

# Encontrar número de asientos menor al promedio 
establishments_chain_seats= establishments_chain_seats[establishments_chain_seats['number'] < 39]
print('Número de establacimientos con menos de 39 asientos:', establishments_chain_seats['number'].count())

Número promedio de asientos: 39.675455286762705
Número de establacimientos con más de 39 asientos: 1131
Número de establacimientos con menos de 39 asientos: 2505


De acuerdo a los resultados se puede comprobar que la característica de las franquicias no es que cuenten con muchos asientos en sus establecimientos. 

In [57]:
#Búsqueda del promedio de número de asientos para cada tipo de restaurante. 
establishments= rest_data.groupby('object_type').agg({'number':'mean'})
establishments=establishments.sort_values(by='number', ascending=False)
establishments

,number
object_type,
Restaurant,48.042316
Bar,44.767123
Fast Food,31.837711
Pizza,28.459375
Cafe,25.000000
Bakery,21.773852


In [58]:
# Gráfico para visualizar el promedio de número de asientos para cada tipo de restaurante. 
labels = establishments.index.to_list()
chain_values = establishments['number'].values

df = px.data.gapminder()
fig = px.bar(df, y=chain_values, x=labels, text_auto='.2s',
            title="Promedio de número de asientos por tipo de establecimiento", 
            color_discrete_sequence=px.colors.sequential.RdBu,
            template='simple_white')

plt.tight_layout()
fig.show()

<Figure size 640x480 with 0 Axes>

Los establecimientos tipo 'Restaurant' tienen el promedio más alto de asientos con 48, en segundo lugar, con poca diferencia continua 'Bar' con 45. En último lugar 'Bakery' con 22 asientos promedio.

In [89]:
#Coloca los datos de los nombres de las calles de la columna address en una columna separada.
rest_data['street_name']= pd.NA

for index, row in rest_data.iterrows():
    address = row['address']

    street_name_match = re.search(r"^(?:\d+)(?:\s+)?(?P<street_name>.+)$", address)

    if street_name_match:
        street_name = street_name_match.group('street_name')
        rest_data.loc[index, 'street_name'] = street_name
    else:
        street_name = "Not Found"

rest_data.head()

,id,object_name,address,chain,object_type,number,street_name
0,11786,HABITAT COFFEE SHOP,3708 N EAGLE ROCK BLVD,False,Cafe,26,N EAGLE ROCK BLVD
1,11787,REILLY'S,100 WORLD WAY 120,False,Restaurant,9,WORLD WAY 120
2,11788,STREET CHURROS,6801 HOLLYWOOD BLVD 253,False,Fast Food,20,HOLLYWOOD BLVD 253
3,11789,TRINITI ECHO PARK,1814 W SUNSET BLVD,False,Restaurant,22,W SUNSET BLVD
4,11790,POLLEN,2100 ECHO PARK AVE,False,Restaurant,20,ECHO PARK AVE


In [90]:
rest_data['street_name'].isna().sum()

np.int64(8)

-Determina el promedio de número de asientos para cada tipo de restaurante. De promedio, ¿qué tipo de restaurante tiene el mayor número de asientos? Traza gráficos.

-Coloca los datos de los nombres de las calles de la columna address en una columna separada.

-Traza un gráfico de las diez mejores calles por número de restaurantes.

-Encuentra el número de calles que solo tienen un restaurante.

-Para las calles con muchos restaurantes, analiza la distribución del número de asientos. ¿Qué tendencias puedes ver?
Llega a una conclusión y aporta recomendaciones sobre el tipo de restaurante y el número de asientos. Comenta la posibilidad de desarrollar una cadena.

Paso 3. Preparar una presentación

Prepara una presentación de tu investigación para compartir con el grupo de inversionistas. Puedes utilizar cualquier herramienta para crearla pero debes convertir tu presentación a formato PDF para la evaluación. Incluye un enlace a la presentación en una celda markdown en el siguiente formato:

Presentation: <enlace al almacenamiento en la nube>